In [1]:
%load_ext autoreload

In [20]:
%autoreload

import numpy as np
from astropy.io import fits
from scipy.interpolate import interp1d
import math as m
import scipy.constants as p
import pandas as pd
import matplotlib.pyplot as plt
import timeit
import sys
sys.path.append("..")
import physics
sys.path.append("../..")
from spec.spectrum import *
#%run 'Electrons_v1.ipynb' might need things form this later
import xsecs
np.set_printoptions(precision=20)

In [56]:
class Rates(object):
    '''
    Allows for easier storage of all xsecs/rates (probabilities)
    Made specifically for ion and heating ONLY.
    
    Attributes
    -----------
    E_spec : Spectrum class
        Electron energy and dNdE spectrum
    n : int
        The bin of interest in the spectrum
    den : ndarray
        density of [H, He, He+] in cm^-3
    rates : dN/dt; Filled with various rates in s^-1
    '''
    
    def __init__(self, E_spec, n, den=[1,1,1], rates=np.zeros(4)):
        '''Return a Rate object with electron of energy *E_spec.eng[n]*
        in a universe with densities *den*; initializes *rate* vector'''
        self.energy = E_spec.eng[n]
        self.n = n
        self.dNdE = E_spec.dNdE[n]
        self.den = den
        self.rates = rates
        self.v = np.sqrt(2*np.array([E_spec.eng[n]])/physics.me)*physics.c 
        self.mult = self.den*self.v
    
    def ion_rate_calc(self):
        '''Fills *self.rate[1:3]* vector by calculating xsec and then converting to rate
        '''
        ion_rates = xsecs.ionize_cs(self.energy*np.ones(3),np.array([1,2,3]))*self.mult
        self.rates = np.append([0], ion_rates)
        return self.rates
    
    def heat_rate_calc(self, x_e, rs):
        '''Fills *self.rate[0]* vector with fraction going to heating
            x_e and rs...
        '''
        dE = xsecs.heating_dE(self.energy, x_e, rs, nH=physics.nH)
        delta_dNdE = np.zeros(len(E_spec.dNdE))
        np.put(delta_dNdE, self.n, self.dNdE)
        delta = Spectrum(E_spec.eng, delta_dNdE, rs)
        shift_delta_eng = E_spec.eng+dE
        delta.shift_eng(shift_delta_eng)
        delta.rebin(E_spec.eng)
        
        heating_frac = delta.dNdE[self.n]/self.dNdE
        self.rates[0] = 1-heating_frac
        return(self.rates)
    def tot_sum(self):
        '''Sums all elements of rate
        '''
        tot_sum = sum(self.rates)
        return tot_sum
    def con_prob(self):
        '''Finds cumulative probability of each process
        '''
        con_prob = self.rates/self.tot_sum()
        return con_prob
    def E_loss(self):
        E_loss_ion=13.6*self.rates[1]
        E_loss_heat=(E_spec.eng[self.n]-E_spec.eng[self.n-1])*self.rates[0]
        E_frac = E_loss_ion/E_loss_heat
        return(E_frac)

In [4]:
eng1 = np.logspace(0.,4.,num = 500)
dNdE1 = np.logspace(0.,5.,num = 500)
rs=1000
E_spec = Spectrum(eng1,dNdE1,rs)
photon_spec = Spectrum(E_spec.eng,np.zeros(len(E_spec.eng)),rs)
TM=np.zeros([len(photon_spec.eng)+2,len(E_spec.eng)]) #possibly make pandas
Transfer_Matrix=pd.DataFrame(TM) #or np.zeros
#print(Transfer_Matrix)
print(np.array([E_spec.eng[330]]))
print(xsecs.ionize_cs(np.array([E_spec.eng[100]]),np.ones(1)))

[ 441.86599564]
[ 0.]


In [5]:
a = Rates(E_spec, 430)
print(a.energy)
print(a.ion_rate_calc())
print(a.heat_rate_calc(0.0001, 300))
print(a.tot_sum())
print(a.con_prob())
print(a.rates)

2798.29687566
[  0.00000000e+00   5.36837606e-19   6.25760697e-19   1.09194109e-19]
[  1.44947387e-11   5.36837606e-19   6.25760697e-19   1.09194109e-19]
1.44947400144e-11
[  9.99999912e-01   3.70367186e-08   4.31715709e-08   7.53336102e-09]
[  1.44947387e-11   5.36837606e-19   6.25760697e-19   1.09194109e-19]


In [46]:
def electron_low_e(E_spec, rs, ion_frac=[10**-4,10**-4,10**-4], den=[1,1,1], dt=1, photon_abscissa=E_spec.eng ,all_outputs=False):
    '''
    Finds transfer matrix for a given initial electron spectrum
    Outputs: photon spectrum, transfer matrix, E_loss to each process
    '''
    #Initialize photon spectrum
    #photon_spec = Spectrum(photon_abscissa,np.zeros(len(E_spec.eng)),rs)
    
    #calculate non-ionized atomic densities
    den[0]=(physics.nH*(1-ion_frac[0]))*(rs)**3
    den[1]=(physics.nHe*(1-ion_frac[1])*(1-ion_frac[2]))*(rs)**3
    den[2]=(physics.nHe*(1-ion_frac[1])*(ion_frac[2]))*(rs)**3 #unsure about how ionfrac works for helium
    
    #ndarray of fractions to heating, ion (H,He,He+)
    frac_table=np.zeros((4,len(E_spec.eng)))
    
    for n in range(len(E_spec.eng)):
        e_rates = Rates(E_spec, n, den=den)
        e_rates.ion_rate_calc()
        e_rates.heat_rate_calc(ion_frac[1], rs)
        #print(e_rates.energy) print(e_rates.rates) print(e_rates.E_loss())
        
    
    

In [57]:
electron_low_e(E_spec,100)

1.0
[  7.91874934290559551187e-05   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
-0.0
1.01862899024
[  7.70251512772768265336e-05   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
1.03760501977
[  7.49218554054742469361e-05   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
1.05693455356
[  7.28759934590650360065e-05   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
1.07662417705
[  7.08859970963704455471e-05   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
1.09668059834
[  6.89503408343172807804e-05   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
1.1171106505
[  6.70675408168674991316e-05   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
1.13792129392
[  6.52361537273327130038e-05   0.000

3.19896891543
[  1.38402191748321357068e-05   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
3.25856247615
[  1.34622904892633954432e-05   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
3.31926620473
[  1.30946817623600608727e-05   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
3.38110078248
[  1.27371111773300071945e-05   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
3.44408727597
[  1.23893046291012609572e-05   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
3.50824714424
[  1.20509955079395325583e-05   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
3.57360224607
[  1.17219244514243925437e-05   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
3.64017484745
[  1.14018392168846460777e-

10.618229411
[  2.28867735052595833167e-06   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
10.8160363031
[  2.22618191958456890234e-06   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
11.0175281379
[  2.16539299913165450562e-06   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
11.2227735621
[  2.10626404850700055249e-06   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
11.4318425013
[  2.04874967835166899022e-06   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
11.6448061837
[  1.99280583512706499505e-06   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
11.8617371645
[  1.93838962991854657503e-06   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
0.0
12.0827093504
[  1.88545931223416118883e-0

35.244730038
[  3.78474965256891948684e-07   3.86649591614647433223e-09
   5.61727879695184189187e-11   0.00000000000000000000e+00]
0.215551889809
35.9013037701
[  3.68140433582020420999e-07   3.93672128289134856077e-09
   5.95069317046382013594e-11   0.00000000000000000000e+00]
0.221501439702
36.5701088078
[  3.58088101393150282092e-07   4.00630455189626266030e-09
   6.28818941002706931394e-11   0.00000000000000000000e+00]
0.227506307982
37.251373008
[  3.48310266184981287552e-07   4.07522285323213461865e-09
   6.62967125029686918148e-11   0.00000000000000000000e+00]
0.233565357509
37.9453284724
[  3.38799424404179205794e-07   4.14345388718533469992e-09
   6.97503620233586568012e-11   0.00000000000000000000e+00]
0.239677438494
38.6522116263
[  3.29548299871085248469e-07   4.21097594201358878584e-09
   7.32417594081015216005e-11   0.00000000000000000000e+00]
0.245841366474
39.3722632996
[  3.20549787491408721962e-07   4.27776791020730366233e-09
   7.67697666709298954047e-11   0.0000000

100.927151463
[  7.81102756919338503394e-08   6.43019947006513706241e-09
   2.69530178337103536896e-10   2.35637449276026057954e-12]
0.606560162967
102.807322383
[  7.59775673486728919670e-08   6.44608182246745037825e-09
   2.72813531980180658207e-10   2.42893048252211361845e-12]
0.613694182948
104.722518989
[  7.39030950880348314058e-08   6.46101119635971478959e-09
   2.76051933880665946554e-10   2.50133694237827804499e-12]
0.620816729296
106.673393773
[  7.18852390946267405525e-08   6.47499636041860673568e-09
   2.79244331501176386066e-10   2.57356732509791102192e-12]
0.627927234913
108.660611385
[  6.99224924627372956820e-08   6.48804634348225333952e-09
   2.82389716579274852143e-10   2.64559506456152385035e-12]
0.635024561693
110.684848855
[  6.80133782626768379487e-08   6.50017042217591319221e-09
   2.85487125666472010441e-10   2.71739361821014496020e-12]
0.642107651267
112.746795824
[  6.61563320791813680444e-08   6.51137810868590444660e-09
   2.88535640573274146742e-10   2.78893

322.862820254
[  1.36563470531925190699e-08   5.99874894759666381571e-09
   3.71917138326956613465e-10   5.77506439186857541837e-12]
1.01175120595
328.877428583
[  1.32835684674148524209e-08   5.97604243654791772564e-09
   3.71904279296716327097e-10   5.80131770543448296888e-12]
1.01725649454
335.004082991
[  1.29209638544125482440e-08   5.95306985606657192868e-09
   3.71850585879728639733e-10   5.82660181086975955989e-12]
1.02273140534
341.244870785
[  1.25682505514035369742e-08   5.92984017171676042194e-09
   3.71756725313335661110e-10   5.85092101760662864511e-12]
1.02817644667
347.601918154
[  1.22251436751596997965e-08   5.90636222145105204533e-09
   3.71623366421377329352e-10   5.87428002677445700674e-12]
1.03359301701
354.077390897
[  1.18914158520055934787e-08   5.88264471505377281854e-09
   3.71451179105147874125e-10   5.89668391875241683032e-12]
1.03897831082
360.673495157
[  1.15668398192880772513e-08   5.85869623368941164870e-09
   3.71240833854743677760e-10   5.91813814071

1013.93945768
[  2.45631759376863101352e-09   4.35693913260482452868e-09
   3.17773490252783774769e-10   5.87623517579495252933e-12]
1.30091995351
1032.82812594
[  2.38929920293173836399e-09   4.33003556082072957762e-09
   3.16372694197291913877e-10   5.85904658991377665063e-12]
1.30484370246
1052.06867102
[  2.32409991252069403345e-09   4.30319781832220763938e-09
   3.14964845933602816189e-10   5.84148774989258152601e-12]
1.30875409549
1071.66764803
[  2.26071961151319555938e-09   4.27642821051896192340e-09
   3.13550289916478970973e-10   5.82356852923470424421e-12]
1.31262293706
1091.63173419
[  2.19904450204921886325e-09   4.24972897361334342403e-09
   3.12129364142125507467e-10   5.80529869940381115768e-12]
1.31648734402
1111.96773112
[  2.13907458412876394505e-09   4.22310227587241325473e-09
   3.10702400205881234622e-10   5.78668792767953311868e-12]
1.32031972826
1132.68256714
[  2.08069661500331903881e-09   4.19655021888857417343e-09
   3.09269723361480031529e-10   5.76774577515

3428.23149373
[  3.96482735531833441200e-10   2.78828552654079172080e-09
   2.21892737369906968177e-10   4.31719628241007991029e-12]
1.5254867793
3492.09598479
[  3.85682707992884843407e-10   2.76830175896975153820e-09
   2.20517923861274930674e-10   4.29152073116641284595e-12]
1.52849039398
3557.15020682
[  3.75223185855588781124e-10   2.74843597462536621131e-09
   2.19148011621673750710e-10   4.26588402143717062961e-12]
1.531296813
3623.41632332
[  3.64991703527550725994e-10   2.72868801625959668294e-09
   2.17783062192878246378e-10   4.24028896220884586019e-12]
1.53432824337
3690.91691066
[  3.55043994204606860876e-10   2.70905771233214152575e-09
   2.16423134872143721232e-10   4.21473828629479188307e-12]
1.53733120641
3759.67496579
[  3.45380724020571960864e-10   2.68954487743436038870e-09
   2.15068286759134298590e-10   4.18923465160542566756e-12]
1.54026704759
3829.71391405
[  3.36001226841631250863e-10   2.67014931270410052479e-09
   2.13718572802127705430e-10   4.16378064241211